In [ ]:
import steno3d
# files can be found at: http://www.tc18.org/code_data_set/3D_images.html
FILENAME = './bonsai.raw' 
FILE_X = 256
FILE_Y = 256
FILE_Z = 256

In [36]:
steno3d.login()


You are already logged in as @michael. To log in as a different user
please `steno3d.logout()`, then login specifying a different
username or API developer key.




In [37]:
import numpy as np
import math


In [38]:
def trilinearInterpolate(xyz, data):
    x0 = math.floor(xyz[0])
    y0 = math.floor(xyz[1])
    z0 = math.floor(xyz[2])
    if (x0 == len(data) - 1): 
        x0 = x0 - 1
    if (y0 == len(data[0]) - 1): 
        y0 = y0 - 1
    if (z0 == len(data[0][0]) - 1): 
        z0 = z0 - 1
        
    x1 = x0 + 1
    y1 = y0 + 1
    z1 = z0 + 1
    
    xd = (xyz[0] - x0) / (x1  - x0)
    yd = (xyz[1] - y0) / (y1  - y0)
    zd = (xyz[2] - z0) / (z1  - z0)
    
    c00 = data[x0, y0, z0] * (1 - xd) + data[x1, y0, z0] * xd
    c01 = data[x0, y0, z1] * (1 - xd) + data[x1, y0, z1] * xd
    c10 = data[x0, y1, z0] * (1 - xd) + data[x1, y1, z0] * xd
    c11 = data[x0, y1, z1] * (1 - xd) + data[x1, y1, z1] * xd
    
    c0 = c00 * (1 - yd) + c10 * yd
    c1 = c01 * (1 - yd) + c11 * yd
    
    c = c0 * (1 - zd) + c1 * zd
    
    return c

In [39]:
data = np.fromfile(FILENAME, '<u1').reshape((FILE_X,FILE_Y,FILE_Z))


In [40]:
num_x = 100
num_y = 100
num_z = 100

data_out = np.zeros(num_x*num_y*num_z)



for i in range(num_x):
    for j in range(num_y):
        for k in range(num_z):
            x = float(i) / (num_z - 1) * (FILE_X - 1)
            y = float(j) / (num_y - 1) * (FILE_Y - 1)
            z = float(k) / (num_z - 1) * (FILE_Z - 1)
            index = i * num_z * num_y + k * num_y + j
            data_out[index] = trilinearInterpolate((x,y,z), data)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [41]:
max_dim = np.max([FILE_X,FILE_Y,FILE_Z])

h1 = np.ones(num_x) * float(FILE_X) / max_dim
h3 = np.ones(num_y) * float(FILE_Y) / max_dim
h2 = np.ones(num_z) * float(FILE_Z) / max_dim

mesh = steno3d.Mesh3DGrid(
    h1=h1, h2=h2, h3=h3
)
volume = steno3d.Volume(
    mesh = mesh,
    project=steno3d.Project(),
    data=dict(
        data=steno3d.DataArray(
            array=data_out,
        ),
        location='CC'
    )
)
volume.project[0].upload()

Verifying your quota for private projects...
Uploading project: 
    Uploading volume: 
        Uploading mesh3dgrid: 
        ... Complete!
        Uploading array: 
        ... Complete!
    ... Complete!
... Complete!
https://steno3d.com/app/Z4FkwDLyWfYkI3t0Nc9G


u'https://steno3d.com/app/Z4FkwDLyWfYkI3t0Nc9G'